In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sympy as sp
from itertools import product


from zgoubidoo.commands import *
from zgoubidoo.fieldmaps import *
from zgoubidoo import ureg as _

# Kinematics and magnet parameters

In [ ]:
kin = zgoubidoo.Kinematics(140 * _.MeV)
B1 = (kin.brho / (1*_.m)) / 50
g_val= 0.15 *_.m

XL = 1 * _.m
WE = 50 * _.degrees
WS = 50 * _.degrees

C0 = 0.1455
C1 = 2.2670
C2 = -0.6395
C3 = 1.1558
C4 = 0
C5 = 0

XE = 2 * g_val
XS = 2 * g_val

In [ ]:
ref_beam = Objet2('BUNCH', BORO=kin.brho)
ref_beam.add(np.array([
    [0, 0, 0, 0, 1.0],
]))
ref_beam;

# Create sympy expressions for the field

## Symbols

In [ ]:
B0 = sp.symbols('B0')
k = sp.symbols('k')
y = sp.symbols('y')
y0 = sp.symbols('y0')
g = sp.symbols('g')
x = sp.symbols('x')

d_e = sp.symbols('d_e')
d_s = sp.symbols('d_s')
xe = sp.symbols('xe')
xs = sp.symbols('xs')
xl = sp.symbols('xl')


C = sp.IndexedBase('C')
L = sp.symbols('L')
i = sp.symbols('i', integer=True)

In [ ]:
# Enge function 
FE = 1 / (1 + sp.exp(sp.Sum(C[i] * (d_e/g)**i,(i,0,5))))
FS = 1 / (1 + sp.exp(sp.Sum(C[i] * (d_s/g)**i,(i,0,5))))

## Definition of the field

In [ ]:
Bx = 0
By = 0
Bz = B0 * FE * FS
Bz

In [ ]:
Bz = Bz.replace(B0, B1.m_as('kG'))

# Usage for Zgoubidoo

## Create the mesh

In [ ]:
x_val = np.linspace(0, XL.m_as('cm') + XE.m_as('cm') + XS.m_as('cm'), 201)
y_val = np.linspace(-20, 20, 101)
z_val = [0]

mesh = np.array(list(product(x_val, y_val, z_val)))

## Create the field map

In [ ]:
parameters_map = {g: g_val.m_as('cm'),
                  C[0]: C0,
                  C[1]: C1,
                  C[2]: C2,
                  C[3]: C3,
                  C[4]: C4,
                  C[5]: C5,
                  d_e: (-(x - xe + y*np.tan(-WE.m_as('radians'))))*np.cos(WE.m_as('radians')),
                  d_s: ((x - xl) - xe + y*np.tan(WS.m_as('radians')))*np.cos(WS.m_as('radians')),
                  B0: B1.m_as('kG'),
                  xe: XE.m_as('cm'),
                  xs: XS.m_as('cm'),
                  xl: XL.m_as('cm'),
                 }
Bz_val = Bz.doit().subs(parameters_map)

In [ ]:
fm = CartesianFieldMap.generate_from_cartesian_expression(bx_expression=Bx,
                                                          by_expression=By,
                                                          bz_expression=Bz_val, 
                                                          mesh=mesh)
fmap = fm(label1='a', path='.', binary=False, MOD=0, MOD2=0, generator=ToscaCartesian2D, load_map=True)

In [ ]:
fmap.length

# Use Zgoubi

In [ ]:
zi = zgoubidoo.Input(name='SPIRAL', line=
               [ref_beam,
                fmap,
               ])
zi.XPAS = 1*_.mm
zi.survey(reference_frame=zgoubidoo.Frame(), with_reference_trajectory=True, reference_kinematics=kin);
zi.IL = 2
zr = zgoubidoo.Zgoubi()(zi).collect()

In [ ]:
artist=zgoubidoo.vis.ZgoubidooPlotlyArtist()

artist.fig['layout']['xaxis']['title'] = 'X (m)'
artist.fig['layout']['yaxis']['title'] = 'Field (T)'
artist.fig['layout']['legend'] = dict(
    x=1.01,
    y=1)

artist.scatter(x=zr.tracks_frenet['X'][::5], 
               y=zr.tracks_frenet['BZ'][::5],
               name='rbend', mode= 'markers')

artist.plot_cartouche(beamline=zi)

artist.render()

In [ ]:
artist=zgoubidoo.vis.ZgoubidooPlotlyArtist(width=800, height=600)

artist.fig['layout']['xaxis']['title'] = 'X (m)'
artist.fig['layout']['yaxis']['title'] = 'Y (m)'
artist.fig['layout']['legend'] = dict(
    x=1.25,
    y=1)


artist.scatter(x=zr.tracks_frenet['X'][::5], 
               y=zr.tracks_frenet['YT'][::5],
               mode= 'markers')

artist.plot_cartouche(beamline=zi)

artist.render()

In [ ]:
artist=zgoubidoo.vis.ZgoubidooPlotlyArtist(width=800, height=600)

artist.fig['layout']['xaxis']['title'] = 'X (m)'
artist.fig['layout']['yaxis']['title'] = 'Y (m)'
artist.fig['layout']['legend'] = dict(
    x=1.25,
    y=1)

artist.scatter(x=zr.tracks_global['XG'][::5], 
               y=zr.tracks_global['YG'][::5],
               mode= 'markers')

artist.plot_beamline(beamline=zi)

artist.render()